In [ ]:
!sudo apt-get install python3.10 python3.10-dev python3.10-distutils
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1

In [ ]:
!git clone https://github.com/neheller/kits23
%cd kits23
!pip3 install -e .

In [ ]:
!kits23_download_data

In [ ]:
!pip install monai==1.3.0 nibabel torch torchvision einops transformers

In [ ]:
import os, glob
from monai.data import Dataset, DataLoader
from monai.transforms import (
    LoadImaged, EnsureChannelFirstd, Spacingd, ScaleIntensityRanged,
    CropForegroundd, Orientationd, ToTensord, Compose
)
from sklearn.model_selection import train_test_split

root = "/content/kits23/dataset"
cases = sorted(glob.glob(os.path.join(root, "case_*")))

data_dicts = [{"image": os.path.join(c, "imaging.nii.gz"), "label": os.path.join(c, "segmentation.nii.gz")} for c in cases]
train_files, val_files = train_test_split(data_dicts, test_size=0.2, random_state=42)

train_transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys=["image", "label"]),
    Orientationd(keys=["image", "label"], axcodes="RAS"),
    Spacingd(keys=["image", "label"], pixdim=(1.5, 1.5, 2.0), mode=("bilinear", "nearest")),
    ScaleIntensityRanged(keys=["image"], a_min=-200, a_max=200, b_min=0.0, b_max=1.0, clip=True),
    CropForegroundd(keys=["image", "label"], source_key="image"),
    ToTensord(keys=["image", "label"])
])

val_transforms = train_transforms

train_ds = Dataset(data=train_files, transform=train_transforms)
val_ds = Dataset(data=val_files, transform=val_transforms)

train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=1, num_workers=2)